In [25]:
!uv pip install praw --system

Using Python 3.11.7 environment at: C:\Users\mwmma\miniconda3
Resolved 9 packages in 4.92s
Prepared 4 packages in 1.97s
Installed 4 packages in 44ms
 + praw==7.8.1
 + prawcore==2.4.0
 + update-checker==0.18.0
 + websocket-client==1.9.0


In [33]:
RELEVANT_SUBREDDITS = [
    "Netherlands",
    "thenetherlands",
    "dutch",
    "Nederland",
    "Netherlands_Memes",
    "NetherlandsHousing",
    "StudyInTheNetherlands",
    "TheHague",
    "Amsterdam",
    "Rentbusters",
    "europe",
]

In [5]:
def get_subreddit_link(name:str) -> str:
    return f"https://www.reddit.com/r/{name}/"

In [ ]:
get_subreddit_link(RELEVANT_SUBREDDITS[2])

'https://www.reddit.com/r/thenetherlands/'

In [50]:
SUBREDDIT = RELEVANT_SUBREDDITS[6]
KEYWORD = "De Broodfabriek"
HEADERS = {"User-Agent": "scraper-(yourname)-v1.0"}
OUTPUT_POSTS = f"datasets/{SUBREDDIT}_{KEYWORD}_posts.csv"
OUTPUT_COMMENTS = f"datasets/{SUBREDDIT}_{KEYWORD}_comments.csv"

RATE_LIMIT = 100          # Reddit cap ≈100 req/min
DELAY = 60 / RATE_LIMIT   # seconds between requests
BACKOFF_TIME = 10         # pause if 429 returned


def safe_request(url, params=None):
    """GET with automatic rate-limit backoff."""
    while True:
        r = requests.get(url, headers=HEADERS, params=params)
        if r.status_code == 429:
            tqdm.write("⏳ Rate limited — backing off for 10 s...")
            time.sleep(BACKOFF_TIME)
            continue
        r.raise_for_status()
        time.sleep(DELAY)  # respect per-request delay
        return r


def fetch_posts(after=None):
    """Fetch subreddit search results (100 per page)."""
    url = f"https://www.reddit.com/r/{SUBREDDIT}/search.json"
    params = {
        "q": KEYWORD,
        "restrict_sr": "on",
        "sort": "new",
        "after": after,
        "limit": 100
    }
    return safe_request(url, params).json()


def fetch_comments(post_id):
    """Fetch comments for a given post."""
    url = f"https://www.reddit.com/r/{SUBREDDIT}/comments/{post_id}.json"
    return safe_request(url, {"limit": 500}).json()

In [51]:
after = None
all_posts = []

print(f"🔍 Searching for posts mentioning '{KEYWORD}' in r/{SUBREDDIT} …\n")

for _ in tqdm(range(50), desc="Fetching posts", ncols=100):
    data = fetch_posts(after)
    posts = data["data"]["children"]
    if not posts:
        break
    all_posts.extend(posts)
    after = data["data"].get("after")
    if not after:
        break

print(f"\n✅ Found {len(all_posts)} posts mentioning {KEYWORD}.\n")

# --- Save posts ---
with open(OUTPUT_POSTS, "w", newline="", encoding="utf-8") as f_posts:
    writer = csv.writer(f_posts)
    writer.writerow(["post_id", "title", "author", "created_utc", "score",
                     "num_comments", "url"])
    for p in all_posts:
        d = p["data"]
        writer.writerow([
            d["id"], d["title"], d.get("author"),
            d["created_utc"], d["score"], d["num_comments"], d["url"]
        ])
print("💾 Posts saved to:", OUTPUT_POSTS)

# --- Fetch comments ---
with open(OUTPUT_COMMENTS, "w", newline="", encoding="utf-8") as f_comments:
    writer = csv.writer(f_comments)
    writer.writerow(["post_id", "comment_id", "author", "created_utc",
                     "body", "score", "parent_id"])

    for p in tqdm(all_posts, desc="Fetching comments", ncols=100):
        post_id = p["data"]["id"]
        try:
            comments_json = fetch_comments(post_id)
            for c in comments_json[1]["data"]["children"]:
                if c["kind"] != "t1":
                    continue
                cd = c["data"]
                writer.writerow([
                    post_id, cd["id"], cd.get("author"), cd["created_utc"],
                    cd["body"].replace("\n", " "),
                    cd["score"], cd["parent_id"]
                ])
        except Exception as e:
            tqdm.write(f"⚠️ Error fetching comments for {post_id}: {e}")

print("✅ Done! Comments saved to:", OUTPUT_COMMENTS)

🔍 Searching for posts mentioning 'De Broodfabriek' in r/StudyInTheNetherlands …



Fetching posts:   0%|                                                        | 0/50 [00:02<?, ?it/s]



✅ Found 0 posts mentioning De Broodfabriek.

💾 Posts saved to: datasets/StudyInTheNetherlands_De Broodfabriek_posts.csv


Fetching comments: 0it [00:00, ?it/s]

✅ Done! Comments saved to: datasets/StudyInTheNetherlands_De Broodfabriek_comments.csv


In [37]:
import pandas as pd
import glob
import os

# --- Find all matching CSV files ---
post_files = glob.glob("*_rijswijk_posts.csv")
comment_files = glob.glob("*_rijswijk_comments.csv")
all_files = post_files + comment_files

if not all_files:
    print("❌ No matching CSV files found.")
    exit()

dfs = []

for file_path in all_files:
    try:
        # Handle malformed rows and quoted text safely
        df = pd.read_csv(
            file_path,
            encoding="utf-8",
            on_bad_lines="skip",      # skip problematic rows
            quoting=1,                # QUOTE_ALL
            engine="python",          # more tolerant
            sep=","
        )
        df["source_file"] = os.path.basename(file_path)
        dfs.append(df)
        print(f"✅ Loaded {file_path} ({len(df)} rows)")
    except Exception as e:
        print(f"⚠️ Error loading {file_path}: {e}")

# --- Combine all ---
combined_df = pd.concat(dfs, ignore_index=True)
print(f"\n📊 Total combined rows: {len(combined_df)}")

# --- Save final combined CSV ---
combined_df.to_csv("combined_rijswijk_data.csv", index=False, encoding="utf-8")
print("💾 Combined CSV saved as 'combined_rijswijk_data.csv'")


✅ Loaded Amsterdam_rijswijk_posts.csv (2 rows)
✅ Loaded dutch_rijswijk_posts.csv (1 rows)
✅ Loaded europe_rijswijk_posts.csv (1 rows)
✅ Loaded Nederland_rijswijk_posts.csv (1 rows)
✅ Loaded NetherlandsHousing_rijswijk_posts.csv (8 rows)
✅ Loaded Netherlands_Memes_rijswijk_posts.csv (0 rows)
✅ Loaded Netherlands_rijswijk_posts.csv (22 rows)
✅ Loaded Rentbusters_rijswijk_posts.csv (2 rows)
✅ Loaded StudyInTheNetherlands_rijswijk_posts.csv (2 rows)
✅ Loaded TheHague_rijswijk_posts.csv (81 rows)
✅ Loaded thenetherlands_rijswijk_posts.csv (21 rows)
✅ Loaded Amsterdam_rijswijk_comments.csv (43 rows)
✅ Loaded dutch_rijswijk_comments.csv (1 rows)
✅ Loaded europe_rijswijk_comments.csv (0 rows)
✅ Loaded Nederland_rijswijk_comments.csv (5 rows)
✅ Loaded NetherlandsHousing_rijswijk_comments.csv (33 rows)
✅ Loaded Netherlands_Memes_rijswijk_comments.csv (0 rows)
✅ Loaded Netherlands_rijswijk_comments.csv (472 rows)
✅ Loaded Rentbusters_rijswijk_comments.csv (2 rows)
✅ Loaded StudyInTheNetherlands_r

C:\Users\mwmma\AppData\Local\Temp\ipykernel_7664\1259660943.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(dfs, ignore_index=True)


In [59]:
import numpy as np

combined_df = pd.read_csv(
    "combined_rijswijk_data.csv",
    encoding="utf-8",
    on_bad_lines="skip",      # skip problematic rows
    quoting=1,                # QUOTE_ALL
    engine="python",          # more tolerant
    sep=","
)


# Create merged 'content' column
combined_df["content"] = (
    combined_df["title"].fillna("") + " " + combined_df["body"].fillna("")
).str.strip()

# Add content_type based on filename pattern in source_file
combined_df["content_type"] = np.where(
    combined_df["source_file"].str.contains("_posts", case=False, na=False),
    "post",
    np.where(
        combined_df["source_file"].str.contains("_comments", case=False, na=False),
        "comment",
        "unknown",
    ),
)

# Create renamed/filtered final DataFrame
cleaned_df = combined_df.rename(columns={"created_utc": "created_at", "source_file": "source"})[
    ["post_id", "created_at", "content", "content_type", "source", "url", "score"]
]
cleaned_df["source"] = "reddit"
# --- Remove empty or duplicate content rows ---
cleaned_df = cleaned_df[cleaned_df["content"].str.strip().astype(bool)]
cleaned_df = cleaned_df.drop_duplicates(subset=["post_id", "content"])

# --- Save to CSV ---
output_path = "cleaned_rijswijk_data.csv"
cleaned_df.to_csv(output_path, index=False, encoding="utf-8")

print(f"✅ Cleaned data saved to: {output_path}")
print(f"📊 Total rows: {len(cleaned_df)}")

✅ Cleaned data saved to: cleaned_rijswijk_data.csv
📊 Total rows: 1460


In [ ]:
SUBREDDIT = RELEVANT_SUBREDDITS[6]
KEYWORD = "De Broodfabriek"
HEADERS = {"User-Agent": "scraper-(yourname)-v1.0"}
OUTPUT_POSTS = f"datasets/{SUBREDDIT}_{KEYWORD}_posts.csv"
OUTPUT_COMMENTS = f"datasets/{SUBREDDIT}_{KEYWORD}_comments.csv"

RATE_LIMIT = 100          # Reddit cap ≈100 req/min
DELAY = 60 / RATE_LIMIT   # seconds between requests
BACKOFF_TIME = 10         # pause if 429 returned


def safe_request(url, params=None):
    """GET with automatic rate-limit backoff."""
    while True:
        r = requests.get(url, headers=HEADERS, params=params)
        if r.status_code == 429:
            tqdm.write("⏳ Rate limited — backing off for 10 s...")
            time.sleep(BACKOFF_TIME)
            continue
        r.raise_for_status()
        time.sleep(DELAY)  # respect per-request delay
        return r


def fetch_posts(after=None):
    """Fetch subreddit search results (100 per page)."""
    url = f"https://www.reddit.com/r/{SUBREDDIT}/search.json"
    params = {
        "q": KEYWORD,
        "restrict_sr": "on",
        "sort": "new",
        "after": after,
        "limit": 100
    }
    return safe_request(url, params).json()


def fetch_comments(post_id):
    """Fetch comments for a given post."""
    url = f"https://www.reddit.com/r/{SUBREDDIT}/comments/{post_id}.json"
    return safe_request(url, {"limit": 500}).json()

In [ ]:
SUBREDDIT = RELEVANT_SUBREDDITS[6]
KEYWORD = "De Broodfabriek"
HEADERS = {"User-Agent": "scraper-(yourname)-v1.0"}
OUTPUT_POSTS = f"datasets/{SUBREDDIT}_{KEYWORD}_posts.csv"
OUTPUT_COMMENTS = f"datasets/{SUBREDDIT}_{KEYWORD}_comments.csv"

RATE_LIMIT = 100          # Reddit cap ≈100 req/min
DELAY = 60 / RATE_LIMIT   # seconds between requests
BACKOFF_TIME = 10         # pause if 429 returned


def safe_request(url, params=None):
    """GET with automatic rate-limit backoff."""
    while True:
        r = requests.get(url, headers=HEADERS, params=params)
        if r.status_code == 429:
            tqdm.write("⏳ Rate limited — backing off for 10 s...")
            time.sleep(BACKOFF_TIME)
            continue
        r.raise_for_status()
        time.sleep(DELAY)  # respect per-request delay
        return r


def fetch_posts(after=None):
    """Fetch subreddit search results (100 per page)."""
    url = f"https://www.reddit.com/r/{SUBREDDIT}/search.json"
    params = {
        "q": KEYWORD,
        "restrict_sr": "on",
        "sort": "new",
        "after": after,
        "limit": 100
    }
    return safe_request(url, params).json()


def fetch_comments(post_id):
    """Fetch comments for a given post."""
    url = f"https://www.reddit.com/r/{SUBREDDIT}/comments/{post_id}.json"
    return safe_request(url, {"limit": 500}).json()

In [ ]:
SUBREDDIT = RELEVANT_SUBREDDITS[6]
KEYWORD = "De Broodfabriek"
HEADERS = {"User-Agent": "scraper-(yourname)-v1.0"}
OUTPUT_POSTS = f"datasets/{SUBREDDIT}_{KEYWORD}_posts.csv"
OUTPUT_COMMENTS = f"datasets/{SUBREDDIT}_{KEYWORD}_comments.csv"

RATE_LIMIT = 100          # Reddit cap ≈100 req/min
DELAY = 60 / RATE_LIMIT   # seconds between requests
BACKOFF_TIME = 10         # pause if 429 returned


def safe_request(url, params=None):
    """GET with automatic rate-limit backoff."""
    while True:
        r = requests.get(url, headers=HEADERS, params=params)
        if r.status_code == 429:
            tqdm.write("⏳ Rate limited — backing off for 10 s...")
            time.sleep(BACKOFF_TIME)
            continue
        r.raise_for_status()
        time.sleep(DELAY)  # respect per-request delay
        return r


def fetch_posts(after=None):
    """Fetch subreddit search results (100 per page)."""
    url = f"https://www.reddit.com/r/{SUBREDDIT}/search.json"
    params = {
        "q": KEYWORD,
        "restrict_sr": "on",
        "sort": "new",
        "after": after,
        "limit": 100
    }
    return safe_request(url, params).json()


def fetch_comments(post_id):
    """Fetch comments for a given post."""
    url = f"https://www.reddit.com/r/{SUBREDDIT}/comments/{post_id}.json"
    return safe_request(url, {"limit": 500}).json()

In [ ]:
SUBREDDIT = RELEVANT_SUBREDDITS[6]
KEYWORD = "De Broodfabriek"
HEADERS = {"User-Agent": "scraper-(yourname)-v1.0"}
OUTPUT_POSTS = f"datasets/{SUBREDDIT}_{KEYWORD}_posts.csv"
OUTPUT_COMMENTS = f"datasets/{SUBREDDIT}_{KEYWORD}_comments.csv"

RATE_LIMIT = 100          # Reddit cap ≈100 req/min
DELAY = 60 / RATE_LIMIT   # seconds between requests
BACKOFF_TIME = 10         # pause if 429 returned


def safe_request(url, params=None):
    """GET with automatic rate-limit backoff."""
    while True:
        r = requests.get(url, headers=HEADERS, params=params)
        if r.status_code == 429:
            tqdm.write("⏳ Rate limited — backing off for 10 s...")
            time.sleep(BACKOFF_TIME)
            continue
        r.raise_for_status()
        time.sleep(DELAY)  # respect per-request delay
        return r


def fetch_posts(after=None):
    """Fetch subreddit search results (100 per page)."""
    url = f"https://www.reddit.com/r/{SUBREDDIT}/search.json"
    params = {
        "q": KEYWORD,
        "restrict_sr": "on",
        "sort": "new",
        "after": after,
        "limit": 100
    }
    return safe_request(url, params).json()


def fetch_comments(post_id):
    """Fetch comments for a given post."""
    url = f"https://www.reddit.com/r/{SUBREDDIT}/comments/{post_id}.json"
    return safe_request(url, {"limit": 500}).json()

In [38]:
# --- Delete original individual CSVs ---
for file_path in all_files:
    try:
        os.remove(file_path)
        print(f"🗑️ Deleted {file_path}")
    except Exception as e:
        print(f"⚠️ Could not delete {file_path}: {e}")

print("\n✅ Cleanup complete! Only 'combined_rijswijk_data.csv' remains.")

🗑️ Deleted Amsterdam_rijswijk_posts.csv
🗑️ Deleted dutch_rijswijk_posts.csv
🗑️ Deleted europe_rijswijk_posts.csv
🗑️ Deleted Nederland_rijswijk_posts.csv
🗑️ Deleted NetherlandsHousing_rijswijk_posts.csv
🗑️ Deleted Netherlands_Memes_rijswijk_posts.csv
🗑️ Deleted Netherlands_rijswijk_posts.csv
🗑️ Deleted Rentbusters_rijswijk_posts.csv
🗑️ Deleted StudyInTheNetherlands_rijswijk_posts.csv
🗑️ Deleted TheHague_rijswijk_posts.csv
🗑️ Deleted thenetherlands_rijswijk_posts.csv
🗑️ Deleted Amsterdam_rijswijk_comments.csv
🗑️ Deleted dutch_rijswijk_comments.csv
🗑️ Deleted europe_rijswijk_comments.csv
🗑️ Deleted Nederland_rijswijk_comments.csv
🗑️ Deleted NetherlandsHousing_rijswijk_comments.csv
🗑️ Deleted Netherlands_Memes_rijswijk_comments.csv
🗑️ Deleted Netherlands_rijswijk_comments.csv
🗑️ Deleted Rentbusters_rijswijk_comments.csv
🗑️ Deleted StudyInTheNetherlands_rijswijk_comments.csv
🗑️ Deleted TheHague_rijswijk_comments.csv
🗑️ Deleted thenetherlands_rijswijk_comments.csv

✅ Cleanup complete! Only '